# The Price is Right

## Week 8 Order of Play

Day 1: Modal.com and SpecialistAgent  
Day 2: RAG, FrontierAgent, Ensemble Agent  
Day 3: ScannerAgent, MessengerAgent  
Day 4: AutonomousPlannerAgent and DealAgentFramework  
Day 5: The Price Is Right Finale

## RAG (Retrieval Augmented Generation) based on a dataset of 800,000 scraped Amazon products

#### For our 2nd agent, we will be asking OpenAI to estimate the price of one of our deals - and we will give it a hand.

We discovered that LLMs are really good at this, out of the box.

And we discovered that we can beat a frontier LLM by fine-tuning an open-source LLM.

Now we are going to try **inference time** techniques instead of training -- by using RAG!

In [2]:
# imports

import os
import logging
from dotenv import load_dotenv
from huggingface_hub import login
import numpy as np
import re
from sentence_transformers import SentenceTransformer
import chromadb
from sklearn.manifold import TSNE
import plotly.graph_objects as go
from litellm import completion
from tqdm.notebook import tqdm
from agents.evaluator import evaluate
from agents.items import Item

In [3]:
# environment

load_dotenv(override=True)
DB = "products_vectorstore"

In [6]:
# Log in to HuggingFace
# If you don't have a HuggingFace account, you can set one up for free at www.huggingface.co
# And then add the HF_TOKEN to your .env file as explained in the project README

hf_token = os.environ['HF_TOKEN']
login(token=hf_token, add_to_git_credential=False)

Note: Environment variable`HF_TOKEN` is set and is the current active token independently from the token you've just configured.


In [7]:
LITE_MODE = True

In [8]:
username = "ed-donner"
dataset = f"{username}/items_lite" if LITE_MODE else f"{username}/items_full"

train, val, test = Item.from_hub(dataset)

print(f"Loaded {len(train):,} training items, {len(val):,} validation items, {len(test):,} test items")

Loaded 20,000 training items, 1,000 validation items, 1,000 test items


# Now create a Chroma Datastore

Now we will use the free, open-source Vector database Chroma.  
We will create a Chroma datastore with 400,000 products from our training dataset.

In [9]:
client = chromadb.PersistentClient(path=DB)

# Introducing the SentenceTransformer Encoding LLM

The all-MiniLM is a very useful model from HuggingFace that maps sentences & paragraphs to 384 dimensional vectors and is ideal for tasks like semantic search.

https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2

It can run pretty quickly locally.

As an alternative, OpenAI provides a closed-source Embeddings model. Benefits compared to OpenAI embeddings:
1. It's free and fast!
3. We can run it locally, so the data never leaves our box - might be useful if you're building a personal RAG

In [10]:
encoder = SentenceTransformer('sentence-transformers/all-MiniLM-L6-v2')

'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 7d8523ee-f10d-4f5f-a27d-0eee7485db0e)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/./config_sentence_transformers.json
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: bc39d004-f5cf-4c05-ada5-4dfe63a1bb57)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/adapter_config.json
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 00feb829-0c26-48fc-aa26-b5f9024100f7)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/1_Pooling/config.json
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSCon

In [11]:
# Pass in a list of texts, get back a numpy array of vectors

vector = encoder.encode(["A proficient AI engineer who has almost reached the finale of AI Engineering Core Track!"])[0]
print(vector.shape)
vector

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(384,)


array([-5.68233989e-02, -6.70465454e-02,  4.41130288e-02,  5.98604837e-03,
       -2.28948891e-02, -2.95300744e-02,  5.56369759e-02,  3.42665948e-02,
       -1.08529419e-01, -3.81690934e-02, -7.43872747e-02, -1.03664294e-01,
        1.69147868e-02,  1.33925583e-03, -6.86191544e-02,  8.99353474e-02,
       -1.45186409e-02, -2.43885424e-02,  4.21855878e-03, -9.62992236e-02,
       -2.51799412e-02,  4.60675955e-02,  4.95350733e-03, -3.88679020e-02,
        1.07715616e-03,  6.82337135e-02, -1.13860266e-02, -5.83415776e-02,
       -1.03801126e-02, -1.74953248e-02, -1.86478682e-02,  4.07063356e-03,
        1.59437452e-02,  6.49721995e-02,  3.71175818e-02,  2.78225299e-02,
       -4.41944785e-02, -2.34373212e-02,  9.71035063e-02, -5.06139509e-02,
       -1.93864703e-02, -3.83471586e-02,  4.76066656e-02, -3.36105898e-02,
        5.08287102e-02,  3.57934758e-02,  2.91821221e-03, -1.06529169e-01,
        4.07211035e-02, -5.85432921e-04, -1.05607405e-01, -1.03584364e-01,
        3.71124186e-02, -

## With that background, let's populate our Chroma database

### By calculating vectors for 800,000 scraped products

This takes 30 minutes on my machine on my GPU - it might take longer for you - feel free to use the Lite dataset!

In [12]:
# Check if the collection exists; if not, create it

collection_name = "products"
existing_collection_names = [collection.name for collection in client.list_collections()]

if collection_name not in existing_collection_names:
    collection = client.create_collection(collection_name)
    for i in tqdm(range(0, len(train), 1000)):
        documents = [item.summary for item in train[i: i+1000]]
        vectors = encoder.encode(documents).astype(float).tolist()
        metadatas = [{"category": item.category, "price": item.price} for item in train[i: i+1000]]
        ids = [f"doc_{j}" for j in range(i, i+1000)]
        ids = ids[:len(documents)]
        collection.add(ids=ids, documents=documents, embeddings=vectors, metadatas=metadatas)

collection = client.get_or_create_collection(collection_name)

# Let's visualize the vectorized data

In [13]:
# It is very fun turning this up to 800_000 and seeing the full dataset visualized,
# but it almost crashes my box every time so do that at your own risk!! 10_000 is safe!

MAXIMUM_DATAPOINTS = 10_000

In [14]:
CATEGORIES = ['Appliances', 'Automotive', 'Cell_Phones_and_Accessories', 'Electronics','Musical_Instruments', 'Office_Products', 'Tools_and_Home_Improvement', 'Toys_and_Games']
COLORS = ['cyan', 'blue', 'brown', 'orange', 'yellow', 'green' , 'purple', 'red']

In [15]:
# Prework
result = collection.get(include=['embeddings', 'documents', 'metadatas'], limit=MAXIMUM_DATAPOINTS)
vectors = np.array(result['embeddings'])
documents = result['documents']
categories = [metadata['category'] for metadata in result['metadatas']]
colors = [COLORS[CATEGORIES.index(c)] for c in categories]

In [39]:
# Let's try a 2D chart
# TSNE stands for t-distributed Stochastic Neighbor Embedding - it's a common technique for reducing dimensionality of data

tsne = TSNE(n_components=2, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

In [40]:
# Create the 2D scatter plot
fig = go.Figure(data=[go.Scatter(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    mode='markers',
    marker=dict(size=4, color=colors, opacity=0.7),
    text=[f"Category: {c}<br>Text: {d[:50]}..." for c, d in zip(categories, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='2D Chroma Vectorstore Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y'),
    width=1200,
    height=800,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [41]:
# Let's try 3D!

tsne = TSNE(n_components=3, random_state=42)
reduced_vectors = tsne.fit_transform(vectors)

In [42]:
# Create the 3D scatter plot
fig = go.Figure(data=[go.Scatter3d(
    x=reduced_vectors[:, 0],
    y=reduced_vectors[:, 1],
    z=reduced_vectors[:, 2],
    mode='markers',
    marker=dict(size=2, color=colors, opacity=0.7),
    text=[f"Category: {c}<br>Text: {d[:50]}..." for c, d in zip(categories, documents)],
    hoverinfo='text'
)])

fig.update_layout(
    title='3D Chroma Vector Store Visualization',
    scene=dict(xaxis_title='x', yaxis_title='y', zaxis_title='z'),
    width=1200,
    height=800,
    margin=dict(r=20, b=10, l=10, t=40)
)

fig.show()

In [43]:
test[0]

<Old Blood Noise Excess V2 Distortion Chorus/Delay Pedal = $219.0>

In [16]:
def vector(item):
    return encoder.encode(item.summary)

In [17]:
def find_similars(item):
    vec = vector(item)
    results = collection.query(query_embeddings=vec.astype(float).tolist(), n_results=5)
    documents = results['documents'][0][:]
    prices = [m['price'] for m in results['metadatas'][0][:]]
    return documents, prices

In [18]:
find_similars(test[0])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

(['Title: Digitech DOD Carcosa Fuzz Distortion Pedal  \nCategory: Electronics  \nBrand: DigiTech  \nDescription: An analog fuzz‑distortion pedal delivering classic to modern tones with standout treble and mid‑range clarity.  \nDetails: Features true‑bypass switching, multiple control knobs (BEFORE, OUTPUT, AFTER, HI‑CUT), a DEMHE‑HALI toggle for low‑mid boost, and a 9‑V power supply.',
  'Title: EarthQuaker Devices Plumes Small Signal Shredder – Purple Sparkle  \nCategory: Guitar Pedals  \nBrand: EarthQuaker Devices  \nDescription: An all‑analog overdrive pedal that delivers three distinct clipping voices and expansive headroom for crystal‑clear, three‑dimensional tone.  \nDetails: Features Flexi‑Switch relay‑based bypass, 9\u202fV power input, and a finely tuned tone control for sculpting low, mid, and high frequencies.',
  'Title: Dunlop Rotovibe Pedal  \nCategory: Audio Effects  \nBrand: Dunlop  \nDescription: A versatile chorus/vibrato pedal that simulates rotating speaker sounds w

In [19]:
# We need to give some context to GPT-5.1 by selecting 5 products with similar descriptions

def make_context(similars, prices):
    message = "For context, here are some other items that might be similar to the item you need to estimate.\n\n"
    for similar, price in zip(similars, prices):
        message += f"Potentially related product:\n{similar}\nPrice is ${price:.2f}\n\n"
    return message

In [48]:
documents, prices = find_similars(test[0])
print(make_context(documents, prices))

For context, here are some other items that might be similar to the item you need to estimate.

Potentially related product:
Title: Digitech DOD Carcosa Fuzz Distortion Pedal  
Category: Electronics  
Brand: DigiTech  
Description: An analog fuzz‑distortion pedal delivering classic to modern tones with standout treble and mid‑range clarity.  
Details: Features true‑bypass switching, multiple control knobs (BEFORE, OUTPUT, AFTER, HI‑CUT), a DEMHE‑HALI toggle for low‑mid boost, and a 9‑V power supply.
Price is $159.99

Potentially related product:
Title: EarthQuaker Devices Plumes Small Signal Shredder – Purple Sparkle  
Category: Guitar Pedals  
Brand: EarthQuaker Devices  
Description: An all‑analog overdrive pedal that delivers three distinct clipping voices and expansive headroom for crystal‑clear, three‑dimensional tone.  
Details: Features Flexi‑Switch relay‑based bypass, 9 V power input, and a finely tuned tone control for sculpting low, mid, and high frequencies.
Price is $99.00


In [20]:
def messages_for(item, similars, prices):
    message = f"Estimate the price of this product. Respond with the price, no explanation\n\n{item.summary}\n\n"
    message += make_context(similars, prices)
    return [{"role": "user", "content": message}]

In [21]:
documents, prices = find_similars(test[0])
print(messages_for(test[0], documents, prices)[0]['content'])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Estimate the price of this product. Respond with the price, no explanation

Title: Excess V2 Distortion/Modulation Pedal  
Category: Music Pedals  
Brand: Old Blood Noise  
Description: A versatile pedal offering distortion and three modulation modes—delay, chorus, and harmonized fifths—with full control over signal routing and expression.  
Details: Features include separate gain, tone, and volume controls; time, depth, and volume per modulation; order switching, soft‑touch bypass, and expression jack for dynamic control.

For context, here are some other items that might be similar to the item you need to estimate.

Potentially related product:
Title: Digitech DOD Carcosa Fuzz Distortion Pedal  
Category: Electronics  
Brand: DigiTech  
Description: An analog fuzz‑distortion pedal delivering classic to modern tones with standout treble and mid‑range clarity.  
Details: Features true‑bypass switching, multiple control knobs (BEFORE, OUTPUT, AFTER, HI‑CUT), a DEMHE‑HALI toggle for low‑

In [22]:
# The function for gpt-5-mini

# def gpt_5__1_rag(item):
#     documents, prices = find_similars(item)
#     response = completion(model="gpt-5.1", messages=messages_for(item, documents, prices), reasoning_effort="none", seed=42)
#     return response.choices[0].message.content

def openRouter8B_rag(item):
    documents, prices = find_similars(item)
    response = completion(
        model="openrouter/meta-llama/llama-3.1-8b-instruct",
        messages=messages_for(item, documents, prices),
       
        seed=42
    )
    return response.choices[0].message.content


In [23]:
# How much does our favorite distortion pedal cost?

test[0].price

219.0

In [24]:
import os
import warnings
import litellm

os.environ["LITELLM_LOG"] = "ERROR"
litellm.set_verbose = False
litellm.suppress_debug_info = True
litellm.drop_params = True

warnings.filterwarnings("ignore")


In [25]:
# Let's do this!!

openRouter8B_rag(test[0])

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

17:14:34 - LiteLLM:INFO: utils.py:3749 - 
LiteLLM completion() model= meta-llama/llama-3.1-8b-instruct; provider = openrouter
17:14:36 - LiteLLM:INFO: utils.py:1575 - Wrapper: Completed Call, calling success_handler


'$189.99'

In [26]:
evaluate(openRouter8B_rag, test)

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

  0%|          | 0/200 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

17:15:26 - LiteLLM:INFO: utils.py:3749 - 
LiteLLM completion() model= meta-llama/llama-3.1-8b-instruct; provider = openrouter
17:15:26 - LiteLLM:INFO: utils.py:1575 - Wrapper: Completed Call, calling success_handler
17:15:26 - LiteLLM:INFO: utils.py:1575 - Wrapper: Completed Call, calling success_handler
17:15:26 - LiteLLM:INFO: utils.py:1575 - Wrapper: Completed Call, calling success_handler


$0 $50 $70 $30 

17:15:26 - LiteLLM:INFO: utils.py:1575 - Wrapper: Completed Call, calling success_handler
17:15:26 - LiteLLM:INFO: utils.py:1575 - Wrapper: Completed Call, calling success_handler


$26 $6 

In [27]:
import modal
Pricer = modal.Cls.from_name("pricer-service", "Pricer")
pricer = Pricer()

In [28]:
def specialist(item):
    return pricer.price.remote(item.summary)


In [29]:
def get_price(reply):
    reply = reply.replace("$", "").replace(",", "")
    match = re.search(r"[-+]?\d*\.\d+|\d+", reply)
    return float(match.group()) if match else 0

## Download the Neural Network weights from Week 6 into this directory

The file `deep_neural_network.pth` here:

https://drive.google.com/drive/folders/1uq5C9edPIZ1973dArZiEO-VE13F7m8MK?usp=drive_link

In [30]:

from agents.deep_neural_network import DeepNeuralNetworkInference

runner = DeepNeuralNetworkInference()
runner.setup()
runner.load("deep_neural_network.pth")

def deep_neural_network(item):
    return runner.inference(item.summary)

INFO:root:Neural Network is using cpu


In [31]:
def ensemble(item):
    price1 = get_price(openRouter8B_rag(item))
    price2 = specialist(item)
    price3 = deep_neural_network(item)
    return price1 * 0.8 + price2 * 0.1 + price3 * 0.1


In [32]:
evaluate(ensemble, test)

  0%|          | 0/200 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

...

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

17:17:39 - LiteLLM:INFO: utils.py:3749 - 
LiteLLM completion() model= meta-llama/llama-3.1-8b-instruct; provider = openrouter
INFO:LiteLLM:
LiteLLM completion() model= meta-llama/llama-3.1-8b-instruct; provider = openrouter
17:17:39 - LiteLLM:INFO: utils.py:3749 - 
LiteLLM completion() model= meta-llama/llama-3.1-8b-instruct; provider = openrouter
INFO:LiteLLM:
LiteLLM completion() model= meta-llama/llama-3.1-8b-instruct; provider = openrouter
17:17:39 - LiteLLM:INFO: utils.py:1575 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
17:17:40 - LiteLLM:INFO: utils.py:1575 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

17:17:41 - LiteLLM:INFO: utils.py:3749 - 
LiteLLM completion() model= meta-llama/llama-3.1-8b-instruct; provider = openrouter
INFO:LiteLLM:
LiteLLM completion() model= meta-llama/llama-3.1-8b-instruct; provider = openrouter
17:17:41 - LiteLLM:INFO: utils.py:1575 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
17:17:41 - LiteLLM:INFO: utils.py:1575 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
17:17:41 - LiteLLM:INFO: utils.py:1575 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
17:17:42 - LiteLLM:INFO: utils.py:1575 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler


$31 $8 $9 $11 $6 $7 $102 $35 $39 $11 $38 $21 $7 

In [33]:
root = logging.getLogger()
root.setLevel(logging.INFO)

In [34]:
from agents.frontier_agent import FrontierAgent

agent = FrontierAgent(collection)
agent.price("Quadcast HyperX condenser mic, connects via usb-c to your computer for crystal clear audio")

INFO:root:[Frontier Agent] Initializing Frontier Agent
INFO:root:[Frontier Agent] Frontier Agent is setting up with Groq
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: f59a3947-6e37-46d9-9b06-8f517f4e1444)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/adapter_config.json
Retrying in 1s [Retry 1/5].
'(ReadTimeoutError("HTTPSConnectionPool(host='huggingface.co', port=443): Read timed out. (read timeout=10)"), '(Request ID: 625b0858-f7eb-486b-a18d-2497a1c0db21)')' thrown while requesting HEAD https://huggingface.co/sentence-transformers/all-MiniLM-L6-v2/resolve/main/config.json
Retrying in 1s [Retry 1/5].
INFO:root:[Frontier Agent] Frontier Agent is ready
I

Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call llama-3.1-8b-instant with context including 5 similar products
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $79.99


79.99

In [35]:
agent.price("Shure MV7+ professional podcaster microphone with usb-c and XLR outputs")

INFO:root:[Frontier Agent] Frontier Agent is performing a RAG search of the Chroma datastore to find 5 similar products


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call llama-3.1-8b-instant with context including 5 similar products
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $299.99


299.99

In [36]:
from agents.neural_network_agent import NeuralNetworkAgent
agent = NeuralNetworkAgent()


INFO:root:[Neural Network Agent] Neural Network Agent is initializing
INFO:root:Neural Network is using cpu
INFO:root:[Neural Network Agent] Neural Network Agent is ready and weights are loaded


In [37]:
agent.price("Shure MV7+ professional podcaster microphone with usb-c and XLR outputs")

INFO:root:[Neural Network Agent] Neural Network Agent is starting a prediction
INFO:root:[Neural Network Agent] Neural Network Agent completed - predicting $162.39


162.39224243164062

In [38]:
from agents.ensemble_agent import EnsembleAgent
agent = EnsembleAgent(collection)

INFO:root:[Ensemble Agent] Initializing Ensemble Agent
INFO:root:[Specialist Agent] Specialist Agent is initializing - connecting to modal
INFO:root:[Frontier Agent] Initializing Frontier Agent
INFO:root:[Frontier Agent] Frontier Agent is setting up with Groq
INFO:sentence_transformers.SentenceTransformer:Use pytorch device_name: cpu
INFO:sentence_transformers.SentenceTransformer:Load pretrained SentenceTransformer: sentence-transformers/all-MiniLM-L6-v2
INFO:root:[Frontier Agent] Frontier Agent is ready
INFO:root:[Neural Network Agent] Neural Network Agent is initializing
INFO:root:Neural Network is using cpu
INFO:root:[Neural Network Agent] Neural Network Agent is ready and weights are loaded
INFO:root:[Ensemble Agent] Ensemble Agent is ready


In [39]:
agent.price("Shure MV7+ professional podcaster microphone with usb-c and XLR outputs")

INFO:root:[Ensemble Agent] Running Ensemble Agent - preprocessing text
17:19:40 - LiteLLM:INFO: utils.py:3749 - 
LiteLLM completion() model= llama3.2; provider = ollama
INFO:LiteLLM:
LiteLLM completion() model= llama3.2; provider = ollama
17:20:04 - LiteLLM:INFO: utils.py:1575 - Wrapper: Completed Call, calling success_handler
INFO:LiteLLM:Wrapper: Completed Call, calling success_handler
INFO:root:[Ensemble Agent] Pre-processed text using ollama/llama3.2
INFO:root:[Specialist Agent] Specialist Agent is calling remote fine-tuned model
INFO:root:[Specialist Agent] Specialist Agent completed - predicting $60.00
INFO:root:[Frontier Agent] Frontier Agent is performing a RAG search of the Chroma datastore to find 5 similar products


Batches:   0%|          | 0/1 [00:00<?, ?it/s]

INFO:root:[Frontier Agent] Frontier Agent has found similar products
INFO:root:[Frontier Agent] Frontier Agent is about to call llama-3.1-8b-instant with context including 5 similar products
INFO:root:[Frontier Agent] Frontier Agent completed - predicting $329.00
INFO:root:[Neural Network Agent] Neural Network Agent is starting a prediction
INFO:root:[Neural Network Agent] Neural Network Agent completed - predicting $296.28
INFO:root:[Ensemble Agent] Ensemble Agent complete - returning $298.83


298.82818603515625